In [1]:
!pwd

/home/mha114/Dropbox/Python/massimal/dataset_specific/20220630_Juvika


In [2]:
import sys
sys.path.append("/massimal/python/tools")
#sys.path.append("/tf/workspace/massimal/python/massimal/tools")

In [3]:
# Imports
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import skimage
# "Local import" from massimal repository (https://github.com/mh-skjelvareid/massimal)
import annotation, image_render, hyspec_io, misc, hyspec_stats, hyspec_cnn

import sklearn.preprocessing
import sklearn.decomposition
import math

In [4]:
tf.config.list_physical_devices('GPU')

2023-02-10 11:27:39.612140: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-10 11:27:39.676908: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-10 11:27:39.677243: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
# Disable GPU (see https://www.tensorflow.org/api_docs/python/tf/config/get_visible_devices )
physical_devices = tf.config.list_physical_devices('GPU')
try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
except:
    # Invalid device or cannot modify virtual devices once initialized.
    print('Invalid device or cannot modify virtual devices once initialized.')
    pass

In [7]:
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [8]:
# Parameters
TILE_SHAPE = (128,128)

In [9]:
# Paths
base_dir = '/media/mha114/Massimal/Bodo_Juvika/Hyperspectral/20220624/'
#base_dir = '/massimal/data/Bodo_Juvika/Hyperspectral/20220624/'

json_gray = base_dir + 'Area/M_Annotation/20220624_Juvika_AreaDetailed_v1/Annotations - grayscale/label_classes.json'
annotation_dir = base_dir + 'Area/M_Annotation/20220624_Juvika_AreaDetailed_v1/Annotations - grayscale'
hyspec_dir = base_dir + 'Area/2_R_rs'
dataset_filename = base_dir + 'Area/M_TensorFlow_Datasets/128-TilesWithWeights/128-TilesWithWeights'

In [10]:
# Find paths to annotated images
ann_file_paths_str = misc.file_pattern_search(annotation_dir,'*.png')
ann_file_paths = [pathlib.Path(fn) for fn in ann_file_paths_str]
for p in ann_file_paths:
    print(p.name)

Juvika_June2022_Pika_L_12-RGB.png
Juvika_June2022_Pika_L_13-RGB.png
Juvika_June2022_Pika_L_16-RGB.png


In [11]:
# Read annotation metadata file, show classes
class_dict = annotation.read_hasty_metadata(json_gray)
for class_name,class_ind in class_dict.items():
    print(f'{class_name}: {class_ind}')

Sand: 1
Seagrass: 2
Rockweed: 3
Blue mussels: 4
Deep water: 5
Turf algae: 6


In [12]:
# Loop through images, spilt into tiles
hyspec_tiles = []
annotation_tiles = []
for annotation_file in ann_file_paths:
    print(f'Processing {annotation_file}')
    # Load hyperspectral image
    hyspec_file = pathlib.Path(hyspec_dir) / (annotation_file.stem[:-3] + 'Crop Wavelengths.bip.hdr')
    hyspec_image, wl, rgb_ind, metadata = hyspec_io.load_envi_image(hyspec_file)
    # Load annotation image
    class_mask = skimage.io.imread(annotation_file)
    class_mask[np.all(hyspec_image==0,axis=2)] = 0        # Don't include zero data
        
    # Extract tiles
    X_tiles, y_tiles = hyspec_cnn.labeled_image_to_tensor_tiles(hyspec_image,class_mask,TILE_SHAPE,padding='VALID')
    hyspec_tiles.append(X_tiles)
    annotation_tiles.append(y_tiles)

# Concatenate into single tensors
hyspec_tiles = tf.concat(hyspec_tiles,axis=0)
annotation_tiles = tf.concat(annotation_tiles,axis=0)

# Show tensor sizes
print(f'{hyspec_tiles.shape=},\n{annotation_tiles.shape=}')

Processing /media/mha114/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/M_Annotation/20220624_Juvika_AreaDetailed_v1/Annotations - grayscale/Juvika_June2022_Pika_L_12-RGB.png


2023-02-10 11:32:02.036643: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-10 11:32:02.037945: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1836000000 exceeds 10% of free system memory.
2023-02-10 11:32:02.869675: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1754726400 exceeds 10% of free system memory.
2023-02-10 11:32:03.694311: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1437204480 exceeds 10% of free system memory.


Processing /media/mha114/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/M_Annotation/20220624_Juvika_AreaDetailed_v1/Annotations - grayscale/Juvika_June2022_Pika_L_13-RGB.png


2023-02-10 11:32:20.251663: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1836000000 exceeds 10% of free system memory.
2023-02-10 11:32:21.075186: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1754726400 exceeds 10% of free system memory.


Processing /media/mha114/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/M_Annotation/20220624_Juvika_AreaDetailed_v1/Annotations - grayscale/Juvika_June2022_Pika_L_16-RGB.png
hyspec_tiles.shape=TensorShape([257, 128, 128, 255]),
annotation_tiles.shape=TensorShape([257, 128, 128])


In [13]:
# Collect random spectra from data for building a PCA model
# (Note: Can proably be optimized, this is just a "quick and dirty" way)
hyspec_samples = []
for tile in hyspec_tiles:
    hyspec_samples.append(hyspec_stats.random_sample_image(np.array(tile)))
hyspec_samples = np.concatenate(hyspec_samples,axis=0)
print(f'{hyspec_samples.shape=}')

hyspec_samples.shape=(209880, 255)


In [14]:
#%% Scaling and PCA model
scaler = sklearn.preprocessing.StandardScaler()
X_sc = scaler.fit_transform(hyspec_samples)

pca = sklearn.decomposition.PCA(n_components=10)   
X_pca = pca.fit_transform(X_sc)

In [15]:
# Show explained variance in PCA model
print(f'Explained variance per component (%): {pca.explained_variance_ratio_*100}')
print(f'Total explained variance: {sum(pca.explained_variance_ratio_)*100:.2f} %')

Explained variance per component (%): [60.92722847 33.92305332  1.7357254   0.76552425  0.33200127  0.19798871
  0.11314546  0.09408659  0.07947655  0.06992978]
Total explained variance: 98.24 %


In [16]:
# PCA transformation of image tiles
pca_tiles = pca.transform(np.reshape(np.array(hyspec_tiles),[-1,hyspec_tiles.shape[-1]]))
pca_tiles = pca_tiles.reshape([*hyspec_tiles.shape[0:3],-1])
print(pca_tiles.shape)

(257, 128, 128, 10)


In [17]:
# Create tensorflow dataset, calculate class weights
n_labeled_pixels = np.count_nonzero(annotation_tiles!=0)
gamma = 0.8
class_weights = {0:0.1}
for class_name, class_index in class_dict.items():
    class_weights[class_index] = (n_labeled_pixels / np.count_nonzero(annotation_tiles==class_index))**gamma
print(class_weights)

{0: 0.1, 1: 2.1529073236778435, 2: 2.4850903217059495, 3: 4.394247481056919, 4: 20.657598875790473, 5: 6.452876590258547, 6: 23.89495425664711}


In [18]:
#sample_weights = np.array(annotation_tiles!=0,dtype=np.float32)  # All annotated pixels get weight 1, background 0
sample_weights = np.zeros_like(annotation_tiles,dtype=np.float32)
for class_ind in class_weights.keys():
    sample_weights[annotation_tiles==class_ind] = class_weights[class_ind]
dataset = tf.data.Dataset.from_tensor_slices((pca_tiles,annotation_tiles,sample_weights))

In [19]:
tf.data.experimental.save(dataset,dataset_filename)

In [20]:
tmp = tf.data.experimental.load(dataset_filename)

257